In [1]:
using MPSKit, MPSKitModels, TensorKit, TensorOperations

In [7]:
function bosonicOperators(cutoff::Int, elt=Float64)
    creadat = zeros(elt, cutoff + 1, cutoff + 1)

    for i in 1:cutoff
        creadat[i + 1, i] = sqrt(i)
    end

    a⁺ = TensorMap(creadat, ℝ^(cutoff + 1), ℝ^(cutoff + 1))
    a⁻ = TensorMap(collect(creadat'), ℝ^(cutoff + 1), ℝ^(cutoff + 1))
    return (a⁺, a⁻)
end

bosonicOperators (generic function with 2 methods)

In [8]:
# define Hamiltionian for infinite basis with chemical potential
function BoseHubbardHamiltonian(t, U, μ, L, M)

    a⁺, a⁻ = bosonicOperators(M) # bosonic operators
    n_op = a⁺ * a⁻

    H_i = TensorMap(zeros, Float64, (ℝ^(M+1))^2 ← (ℝ^(M+1))^2)
    H_i += -t * (a⁺ ⊗ a⁻ + a⁻ ⊗ a⁺) + U/2 * n_op * (n_op - id(ℝ^(M+1))) ⊗ id(ℝ^(M+1))
    H_i += -μ * n_op ⊗ id(ℝ^(M+1))

    return @mpoham sum(H_i{i,i+1} for i in (1:L))
end

BoseHubbardHamiltonian (generic function with 1 method)

In [47]:
H = BoseHubbardHamiltonian(0.001, 4, 0, 10, 5)

MPOHamiltonian{CartesianSpace, TensorMap{CartesianSpace, 2, 2, Trivial, Matrix{Float64}, Nothing, Nothing}, Float64}(MPSKit.SparseMPOSlice{CartesianSpace, TensorMap{CartesianSpace, 2, 2, Trivial, Matrix{Float64}, Nothing, Nothing}, Float64}[[TensorMap((ℝ^1 ⊗ ℝ^6) ← (ℝ^6 ⊗ ℝ^1)):
[:, :, 1, 1] =
 1.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.0  1.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 0.0  0.0  1.0  0.0  0.0  0.0

[:, :, 4, 1] =
 0.0  0.0  0.0  1.0  0.0  0.0

[:, :, 5, 1] =
 0.0  0.0  0.0  0.0  1.0  0.0

[:, :, 6, 1] =
 0.0  0.0  0.0  0.0  0.0  1.0
 TensorMap((ℝ^1 ⊗ ℝ^6) ← (ℝ^6 ⊗ ℝ^3)):
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 0.0  0.0  -0.08276058886023684  0.0  0.0  0.0

[:, :, 4, 1] =
 0.0  0.0  0.0  -0.24828176658071027  0.0  0.0

[:, :, 5, 1] =
 0.0  0.0  0.0  0.0  -0.4965635331614207  0.0

[:, :, 6, 1] =
 0.0  0.0  0.0  0.0  0.0  -0.8276058886023683

[:, :, 1, 2] =
 -6.63392351284039e-26  0.25819888974698346  0.0 

In [54]:
state = FiniteMPS(rand, Float64, 10, ℝ^(5+1), ℝ^40)

FiniteMPS{TensorMap{CartesianSpace, 2, 1, Trivial, Matrix{Float64}, Nothing, Nothing}, TensorMap{CartesianSpace, 1, 1, Trivial, Matrix{Float64}, Nothing, Nothing}}(Union{Missing, TensorMap{CartesianSpace, 2, 1, Trivial, Matrix{Float64}, Nothing, Nothing}}[TensorMap((ℝ^1 ⊗ ℝ^6) ← ProductSpace(ℝ^6)):
[:, :, 1] =
 0.396445978629671  0.14492883147006794  …  0.13939421910191055

[:, :, 2] =
 -0.16341211492521165  0.1359723823082457  …  0.7975826019934606

[:, :, 3] =
 0.1015678486032256  0.09778243980006115  …  0.2736117507950796

[:, :, 4] =
 0.7711912737629651  0.455022578096343  …  -0.02831339126122001

[:, :, 5] =
 0.444148379244126  -0.8400469960083706  …  0.27232604444436703

[:, :, 6] =
 -0.11750436307139983  0.19550252541785368  …  0.4411402632924528
, TensorMap((ℝ^6 ⊗ ℝ^6) ← ProductSpace(ℝ^36)):
[:, :, 1] =
  0.41940866152178735    0.3773562756016463    …   0.2851991854764523
  0.14325664700764823    0.10922029620726897       0.057630017008184334
  0.17099352077733745    0.13502043

In [55]:
(groundstate,_) = find_groundstate(state, H, DMRG(tol=1e-10, maxiter=200));

┌ Info: Iteraton 0 error 0.04157500064109847
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 1 error 0.0002733762925958449
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 2 error 4.193040418691076e-6
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 3 error 1.024284927706753e-7
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 4 error 1.3667262445244702e-9
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 5 error 3.6624106139623903e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 6 error 1.4605211249087186e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 7 error 1.0714578121228833e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 8 error 9.269403196372033e-11
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


In [56]:
E0 = sum(expectation_value(groundstate, H))

-0.007223688606582269

In [57]:
a⁺, a⁻ = bosonicOperators(5) # bosonic operators
n_op = a⁺ * a⁻

TensorMap(ProductSpace(ℝ^6) ← ProductSpace(ℝ^6)):
 0.0  0.0  0.0                 0.0                 0.0  0.0
 0.0  1.0  0.0                 0.0                 0.0  0.0
 0.0  0.0  2.0000000000000004  0.0                 0.0  0.0
 0.0  0.0  0.0                 2.9999999999999996  0.0  0.0
 0.0  0.0  0.0                 0.0                 4.0  0.0
 0.0  0.0  0.0                 0.0                 0.0  5.000000000000001


In [58]:
N_b = expectation_value(groundstate, n_op)

10-element Vector{Float64}:
 0.5282471343243377
 0.4998171978348426
 0.5291458742511519
 0.49871086729736636
 0.5306077998562861
 0.4969335911390502
 0.5322375376490353
 0.49833588434479004
 0.5023383031724719
 4.38362581013067

In [59]:
sum(N_b)

9.000000000000002